In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Rectal_Cancer"
cohort = "GSE119409"

# Input paths
in_trait_dir = "../../input/GEO/Rectal_Cancer"
in_cohort_dir = "../../input/GEO/Rectal_Cancer/GSE119409"

# Output paths
out_data_file = "../../output/preprocess/Rectal_Cancer/GSE119409.csv"
out_gene_data_file = "../../output/preprocess/Rectal_Cancer/gene_data/GSE119409.csv"
out_clinical_data_file = "../../output/preprocess/Rectal_Cancer/clinical_data/GSE119409.csv"
json_path = "../../output/preprocess/Rectal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data from rectal cancer"
!Series_summary	"A supervised method (Significance Analysis of Microarrays -SAM-) was used to find statistically significance (adjusted p<0.05) in differentially expressed genes between responding and non-responding groups."
!Series_overall_design	"To further investigate the correlation between gene expression and response to neoadjuvant radiotherapy, mRNA expression in pre-therapy biopsies was profiled into responding and non-responding groups."
Sample Characteristics Dictionary:
{0: ['disease state: rectal cancer'], 1: ['tissue: rectal cancer biopsy'], 2: ['sensitivity: sensitive', 'sensitivity: unknown', 'sensitivity: resistant'], 3: ['patient age: 52', 'patient age: 57', 'patient age: 65', 'patient age: 61', 'patient age: 62', 'patient age: 58', 'patient age: 63', 'patient age: 70', 'patient age: 74', 'patient age: 72', 'patient age: 51', 'patient age: 45', 'patient age: 77', 'patient age: 64', 'patient age:

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import json
from typing import Optional, Dict, Any, Callable

# 1. Gene Expression Data Availability
# Based on background info, this appears to be mRNA expression data, so gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Trait data - Sensitivity to therapy (responding vs non-responding to radiotherapy)
trait_row = 2  # "sensitivity" row in sample characteristics

# Convert trait values (sensitivity to therapy)
def convert_trait(value):
    if not isinstance(value, str):
        return None
    
    value = value.lower().strip()
    if 'sensitivity:' in value:
        value = value.split('sensitivity:')[1].strip()
    
    if value == 'sensitive' or value == 'responding':
        return 1
    elif value == 'resistant' or value == 'non-responding':
        return 0
    else:
        return None  # For 'unknown' or other values

# 2.2 Age data
age_row = 3  # "patient age" row in sample characteristics

# Convert age values
def convert_age(value):
    if not isinstance(value, str):
        return None
    
    value = value.lower().strip()
    if 'patient age:' in value:
        value = value.split('patient age:')[1].strip()
    
    if value == 'unknown':
        return None
    
    try:
        return float(value)  # Age as continuous value
    except (ValueError, TypeError):
        return None

# 2.3 Gender data - Not available in the sample characteristics
gender_row = None  # No gender information in the data

def convert_gender(value):
    # Function defined but not used since gender data is not available
    return None

# 3. Save Metadata - Initial filtering on usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait data is available)
if trait_row is not None:
    try:
        # Extract clinical features using the clinical_data variable that should be available
        # from a previous step (not loading from file)
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,  # Use existing clinical_data variable
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(clinical_features)
        print("Clinical Features Preview:")
        print(preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save clinical features to CSV
        clinical_features.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical features saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error in clinical feature extraction: {e}")
        # If an error occurs, still ensure we have a valid clinical data file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        pd.DataFrame(columns=[trait, 'Age']).to_csv(out_clinical_data_file, index=False)


Clinical Features Preview:
{'GSM3374350': [1.0, 52.0], 'GSM3374351': [nan, 57.0], 'GSM3374352': [1.0, 65.0], 'GSM3374353': [0.0, 61.0], 'GSM3374354': [0.0, 62.0], 'GSM3374355': [0.0, 58.0], 'GSM3374356': [1.0, 63.0], 'GSM3374357': [0.0, 70.0], 'GSM3374358': [0.0, 61.0], 'GSM3374359': [0.0, 74.0], 'GSM3374360': [0.0, 72.0], 'GSM3374361': [0.0, 51.0], 'GSM3374362': [1.0, 70.0], 'GSM3374363': [0.0, 45.0], 'GSM3374364': [0.0, 77.0], 'GSM3374365': [0.0, 64.0], 'GSM3374366': [1.0, 66.0], 'GSM3374367': [0.0, 43.0], 'GSM3374368': [1.0, 65.0], 'GSM3374369': [1.0, 51.0], 'GSM3374370': [1.0, 66.0], 'GSM3374371': [0.0, 52.0], 'GSM3374372': [0.0, 39.0], 'GSM3374373': [0.0, 72.0], 'GSM3374374': [0.0, 71.0], 'GSM3374375': [0.0, 35.0], 'GSM3374376': [0.0, 61.0], 'GSM3374377': [0.0, 45.0], 'GSM3374378': [0.0, 42.0], 'GSM3374379': [0.0, 56.0], 'GSM3374380': [0.0, 40.0], 'GSM3374381': [0.0, 62.0], 'GSM3374382': [0.0, 67.0], 'GSM3374383': [nan, 63.0], 'GSM3374384': [0.0, 70.0], 'GSM3374385': [nan, 63.0], 

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers in the gene expression data are in a format like '1007_s_at', '1053_at', etc.
# These appear to be Affymetrix probe IDs, not human gene symbols.
# Affymetrix IDs need to be mapped to standard gene symbols for proper analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe the gene identifiers in the gene expression data and the gene annotation data
# The gene identifiers in the gene expression data are probe IDs like '1007_s_at'
# In the gene annotation data, the 'ID' column contains these probe IDs
# The 'Gene Symbol' column contains the corresponding gene symbols

# 2. Get gene mapping dataframe by extracting the probe ID and gene symbol columns
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the first few rows of the gene expression data to verify
print("First few gene symbols after mapping:")
print(gene_data.index[:10])


First few gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Get clinical data from previous step
# Load the clinical features from the saved file
clinical_file_path = out_clinical_data_file
if os.path.exists(clinical_file_path):
    clinical_features = pd.read_csv(clinical_file_path)
    print(f"Clinical features loaded from {clinical_file_path}")
    print(f"Clinical features shape: {clinical_features.shape}")
else:
    # If file doesn't exist, we need to extract it again
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=2,
        convert_trait=convert_trait,
        age_row=3,
        convert_age=convert_age,
        gender_row=None,
        convert_gender=None
    )
    print(f"Clinical features re-extracted")
    print(f"Clinical features shape: {clinical_features.shape}")

# 2. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features.T, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(f"First few columns: {list(linked_data.columns[:5])}")

# Check what columns are available in the linked data
print(f"Columns in linked_data: {list(linked_data.columns[:5])}")

# 3. Handle missing values in the linked data
# Find the correct trait column name (it should be the first column)
trait_column = linked_data.columns[0]
print(f"Using trait column: {trait_column}")

linked_data_processed = handle_missing_values(linked_data, trait_column)
print(f"Shape after handling missing values: {linked_data_processed.shape}")

# Add validation check - if no samples remain, note the issue
if linked_data_processed.shape[0] == 0:
    print("No samples remain after handling missing values. The dataset cannot be processed further.")
    is_trait_biased = True  # Mark as biased since we can't use it
    unbiased_linked_data = linked_data_processed
else:
    # 4. Determine whether the trait and demographic features are severely biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_processed, trait_column)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from rectal cancer patients with treatment response data (sensitive/resistant)."
)

# 6. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Normalized gene data shape: (19845, 66)
First few normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']


Normalized gene data saved to ../../output/preprocess/Rectal_Cancer/gene_data/GSE119409.csv
Clinical features loaded from ../../output/preprocess/Rectal_Cancer/clinical_data/GSE119409.csv
Clinical features shape: (2, 66)
Linked data shape: (68, 19911)
First few columns: ['GSM3374350', 'GSM3374351', 'GSM3374352', 'GSM3374353', 'GSM3374354']
Columns in linked_data: ['GSM3374350', 'GSM3374351', 'GSM3374352', 'GSM3374353', 'GSM3374354']
Using trait column: GSM3374350
Shape after handling missing values: (2, 55)
For the feature 'GSM3374350', the least common label is '1.0' with 1 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'GSM3374350' in this dataset is fine.

Linked data saved to ../../output/preprocess/Rectal_Cancer/GSE119409.csv
